In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.11 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

2024-06-24 04:17:04.257706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 04:17:04.257833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 04:17:04.418970: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
from datasets import load_dataset

dataset_name = "Sahi19/IndianLawComplete"
dataset = load_dataset(dataset_name, split="train")


In [11]:
dataset[1]

{'text': '<s>[INST]Can you explain Section 2 regarding the punishment of offenses committed within India?. [/INST]Section 2 explains that every person shall be liable to punishment under this Code for every act or omission contrary to its provisions committed within India. [Chapter 1, Section 2]</s> '}

In [14]:

def format_example(example):
    text = example['text']
    user_prompt = text.split("[INST]")[1].split("[/INST]")[0].strip()
    assistant_response = text.split("[/INST]")[1].strip()
    formatted_text = f"<s>[INST] <<USER: {user_prompt}>> [/INST] <<ASSISTANT: {assistant_response}>> </s>"
    return {"text": formatted_text}

formatted_dataset = dataset.map(format_example, remove_columns=dataset.column_names)

for i in range(3):
    print(formatted_dataset[i]['text'])

formatted_dataset.save_to_disk("formatted_dataset")


Map:   0%|          | 0/13133 [00:00<?, ? examples/s]

<s>[INST] <<USER: What is the main purpose of the Indian Penal Code, and where can I find this information?.>> [/INST] <<ASSISTANT: The main purpose of the Indian Penal Code is outlined in Section 1 of Chapter 1, which is the 'Introduction' chapter. It states that this Code shall be called the Indian Penal Code and shall extend to the whole of India except the State of Jammu and Kashmir. [Chapter 1, Section 1]</s>>> </s>
<s>[INST] <<USER: Can you explain Section 2 regarding the punishment of offenses committed within India?.>> [/INST] <<ASSISTANT: Section 2 explains that every person shall be liable to punishment under this Code for every act or omission contrary to its provisions committed within India. [Chapter 1, Section 2]</s>>> </s>
<s>[INST] <<USER: What does Section 3 say about punishment for offenses committed beyond India?.>> [/INST] <<ASSISTANT: Section 3 states that the provisions of this Code apply to offenses committed by any citizen of India in any place without and beyon

Saving the dataset (0/1 shards):   0%|          | 0/13133 [00:00<?, ? examples/s]

In [18]:
dataset=formatted_dataset

In [15]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use

# Fine-tuned model name
new_model = "Llama-2-7b-law-finetuned"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load dataset (you can process it here)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)

In [21]:
#TO GET OUTPUT USING OUR MODEL 
from transformers import AutoTokenizer, AutoModelForCausalLM

new_model_path = "Llama-2-7b-law-finetuned"

tokenizer = AutoTokenizer.from_pretrained(new_model_path)
model = AutoModelForCausalLM.from_pretrained(new_model_path)


def generate_response(query, max_length=100):
    inputs = tokenizer("<s>[INST] <<USER: " + query + " >> [/INST]", return_tensors="pt")
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    # Generate response
    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_length)

    # Decode the generated output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


query = "What is the main purpose of the Indian Penal Code?"
output = generate_response(query)

print("User Query:", query)
print("LLaMA 2 Response:", output)


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: Llama-2-7b-law-finetuned is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.